In [25]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import torch

In [2]:
posts = pd.read_csv(r'C:\Users\Aarush Kumar\Downloads\Recommendation\df_posts.csv')
likes = pd.read_csv(r'C:\Users\Aarush Kumar\Downloads\Recommendation\posts_likes.csv')
comments = pd.read_csv(r'C:\Users\Aarush Kumar\Downloads\Recommendation\post_comments.csv')
grps = pd.read_csv( r'C:\Users\Aarush Kumar\Downloads\Recommendation\grp.csv')
org = pd.read_csv( r'C:\Users\Aarush Kumar\Downloads\Recommendation\org.csv')
user = pd.read_csv( r'C:\Users\Aarush Kumar\Downloads\Recommendation\user.csv')

In [3]:
likes.columns

Index(['react_type', 'member_id', 'source_id', 'source_type', 'created_at',
       'deleted_at'],
      dtype='object')

In [4]:
posts['Engagement_Rate']

0         20.689655
1        100.000000
2          0.000000
3          0.000000
4          0.000000
            ...    
14917     28.571429
14918      0.000000
14919    200.000000
14920    200.000000
14921     50.000000
Name: Engagement_Rate, Length: 14922, dtype: float64

In [5]:
zeros_count = (posts['Engagement_Rate'] == 0).sum()
print(zeros_count)

6599


In [6]:
posts_with_org_country = posts.merge(org[['id', 'country']], 
                                        left_on=['association_id'], 
                                        right_on=['id'], 
                                        how='left')

In [7]:
posts_with_org_and_grp_country = posts_with_org_country.merge(grps[['id', 'organization_id']].merge(org[['id', 'country']], on='id'), 
                                                             left_on=['association_id', 'association_type'], 
                                                             right_on=['id', 'organization_id'], 
                                                             how='left')

In [8]:
def fill_country(row):
    if row['association_type'] == 'organization':
        return row['country_x']
    elif row['association_type'] == 'group':
        return row['country_y']

In [9]:
posts_with_org_and_grp_country['association_country'] = posts_with_org_and_grp_country.apply(fill_country, axis=1)

In [10]:
posts_with_org_and_grp_country.drop(columns=['country_x', 'country_y', 'organization_id'], inplace=True)

In [11]:
posts_1 = posts_with_org_and_grp_country[['id_x','Engagement_Rate','description','association_type','association_country']]
explicit_rating = pd.merge(likes,posts_1,how='inner', left_on='source_id', right_on='id_x')

In [12]:
posts_1['description'][1]

'Praise the Lord'

In [14]:
rating = explicit_rating[['description','member_id','Engagement_Rate','source_id','react_type','association_type','association_country']]
rating.loc[:, 'association_country'] = rating['association_country'].astype(str)
post = posts[['id']]

C:\Users\Aarush Kumar\AppData\Local\Temp\ipykernel_4932\577771061.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating.loc[:, 'association_country'] = rating['association_country'].astype(str)


In [15]:
explicit_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28116 entries, 0 to 28115
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   react_type           28116 non-null  object 
 1   member_id            28116 non-null  object 
 2   source_id            28116 non-null  object 
 3   source_type          28116 non-null  object 
 4   created_at           28116 non-null  object 
 5   deleted_at           553 non-null    object 
 6   id_x                 28116 non-null  object 
 7   Engagement_Rate      28116 non-null  float64
 8   description          28116 non-null  object 
 9   association_type     28116 non-null  object 
 10  association_country  27697 non-null  object 
dtypes: float64(1), object(10)
memory usage: 2.6+ MB


In [16]:
from transformers import AutoModel, AutoTokenizer

In [17]:
from transformers import AlbertTokenizer

In [18]:
tokenizer = AlbertTokenizer.from_pretrained('ai4bharat/indic-bert')

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

C:\Users\Aarush Kumar\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Aarush Kumar\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

In [19]:
model = AutoModel.from_pretrained('ai4bharat/indic-bert')

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

C:\Users\Aarush Kumar\anaconda3\Lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [20]:
org_sentences = org['title'].tolist()
post_sentences = posts['description'].tolist()
comment_sentences = comments['description'].tolist()

In [21]:
all_sentences = org_sentences + post_sentences + comment_sentences

In [22]:
from transformers import AlbertTokenizer, AutoModel
sentences = all_sentences

tokenizer = AlbertTokenizer.from_pretrained('ai4bharat/indic-bert')
model = AutoModel.from_pretrained('ai4bharat/indic-bert')
word_vectors = []
for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    word_vectors.append(outputs.last_hidden_state.mean(dim=1).detach().numpy())

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [23]:
len(all_sentences)

15934

In [26]:
text = " FMG 216 இனி பாவம் செய்யாதே."
inputs = tokenizer(text,max_length = 30, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    print(outputs)
last_hidden_states = outputs.last_hidden_state

BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.0121, -0.0035, -0.0175,  ..., -0.0262, -0.0208, -0.0218],
         [ 0.2402, -0.3323, -0.2352,  ..., -0.0267, -0.0873, -0.1347],
         [ 0.5314, -0.3854,  0.0146,  ...,  0.1481,  0.3342, -0.2639],
         ...,
         [ 1.0052, -0.2547, -0.4842,  ...,  0.1704, -0.0805,  0.1340],
         [-0.2866, -0.1035,  0.1761,  ..., -0.2732, -0.1044,  0.1791],
         [-0.0121, -0.0035, -0.0175,  ..., -0.0262, -0.0208, -0.0218]]]), pooler_output=tensor([[-0.0506,  0.0527,  0.0218,  0.0023,  0.0487,  0.0852,  0.0380, -0.0052,
         -0.0145,  0.0635, -0.0134,  0.0493,  0.0122, -0.0192,  0.0493, -0.0149,
         -0.0429,  0.0018, -0.1177, -0.0836,  0.0774,  0.0747, -0.0347, -0.1007,
          0.0065, -0.0344, -0.0557, -0.0025,  0.0232,  0.0092,  0.0275, -0.0026,
          0.0171,  0.0373,  0.0025, -0.0405, -0.0199, -0.0255,  0.1135,  0.0268,
          0.0294, -0.0142, -0.1132,  0.0402, -0.0674, -0.0349,  0.0237, -0.0666,
         -0.

In [27]:
encoded_sentences = tokenizer(all_sentences, padding=True, max_length = 30, truncation=True, return_tensors="pt")

In [28]:
encoded_sentences

{'input_ids': tensor([[    2, 77146,  2444,  ...,     0,     0,     0],
        [    2, 27889,    32,  ...,     0,     0,     0],
        [    2,    33, 44218,  ...,     0,     0,     0],
        ...,
        [    2, 47525, 48729,  ...,     0,     0,     0],
        [    2,    26,  2604,  ...,     0,     0,     0],
        [    2, 26695,  2052,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [30]:
!pip install tensorflow-recommenders

  Obtaining dependency information for tensorflow-recommenders from https://files.pythonhosted.org/packages/d3/91/7f9977f26bc0c94269d3f157710e9f1a112d1af23d4588285d846228ce3c/tensorflow_recommenders-0.7.3-py3-none-any.whl.metadata
  Obtaining dependency information for typing-extensions<4.6.0,>=3.6.6 from https://files.pythonhosted.org/packages/31/25/5abcd82372d3d4a3932e1fa8c3dbf9efac10cc7c0d16e78467460571b404/typing_extensions-4.5.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/96.2 kB ? eta -:--:--
   ---------------------------------------- 96.2/96.2 kB 5.4 MB/s eta 0:00:00
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.10.0
    Uninstalling typing_extensions-4.10.0:
      Successfully uninstalled typing_extensions-4.10.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.104.0 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.
pydantic 2.4.2 requires typing-extensions>=4.6.1, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.10.1 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.


In [31]:
import os
import tempfile
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

In [32]:
ratings = tf.data.Dataset.from_tensor_slices({
    'react_type': rating['react_type'],
    'user_id': rating['member_id'],
    'source_id': rating['source_id'],
    'country' : rating['association_country'],
})
posts = tf.data.Dataset.from_tensor_slices(post['id'])

In [33]:
post_description = posts.batch(1_000)
unique_post_descriptions = np.unique(np.concatenate(list(post_description)))

user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

reaction_type = ratings.batch(1_000_000).map(lambda x: x["react_type"])
unique_reaction_types = np.unique(np.concatenate(list(reaction_type)))
country_type = ratings.batch(1_000_000).map(lambda x: x["country"])
unique_country_types = np.unique(np.concatenate(list(reaction_type)))

In [34]:
class deep_UserModel(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.user_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),  # Adjust dimension to 64
        ])

        
        self.reaction_embedding = tf.keras.Sequential([
                tf.keras.layers.StringLookup(
                    vocabulary=unique_reaction_types, mask_token=None),
                tf.keras.layers.Embedding(len(unique_reaction_types) + 1, 32),  # Keep the dimension as 32
            ])
        
        self.country_embedding = tf.keras.Sequential([
                tf.keras.layers.StringLookup(
                    vocabulary=unique_country_types, mask_token=None),
                tf.keras.layers.Embedding(len(unique_reaction_types) + 1, 32),  # Keep the dimension as 32
            ])

    def call(self, inputs):
        return tf.concat([
            self.user_embedding(inputs["user_id"]),
            self.reaction_embedding(inputs["react_type"]),
            self.country_embedding(inputs['country']),
        ], axis=1)

In [39]:
class QueryModel(tf.keras.Model):
  """Model for encoding queries."""
  def __init__(self, layer_sizes):
    """Model for encoding queries.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()
    self.embedding_model = deep_UserModel()
    self.dense_layers = tf.keras.Sequential()
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [40]:
max_tokens = 10000  
title_vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_tokens)
title_vectorizer.adapt(posts)

In [41]:
class deep_Model(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.title_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_post_descriptions, mask_token=None),
            tf.keras.layers.Embedding(len(unique_post_descriptions) + 1, 32)
        ])

        self.title_text_embedding = tf.keras.Sequential([
            title_vectorizer,
            tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
            tf.keras.layers.GlobalAveragePooling1D(),
        ])
        
    def call(self, titles):
        return tf.concat([
            self.title_embedding(titles),
            self.title_text_embedding(titles),
        ], axis=1)

In [42]:
class UserModel(tf.keras.Model):
  """Model for encoding."""
  def __init__(self, layer_sizes):
    """Model for encoding.
    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()
    self.embedding_model = deep_PostModel()
    self.dense_layers = tf.keras.Sequential()
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [43]:
class deep_postsearchModel(tfrs.models.Model):
    def __init__(self, layer_sizes):
        super().__init__()
        self.query_model = QueryModel(layer_sizes)
        self.candidate_model = CandidateModel(layer_sizes)
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=posts.batch(128).map(lambda x: self.candidate_model(x)),
            ),
        )
    def compute_loss(self, features, training=False):
        query_embeddings = self.query_model({
            "user_id": features["user_id"],
            "react_type": features["react_type"],
            "country": features['country'],# Assuming "react_type" is provided
        })
        post_embeddings = self.candidate_model(features["source_id"])
        return self.task(query_embeddings, post_embeddings)

In [44]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)
train_size = int(len(ratings) * 0.8)
test_size = len(ratings) - train_size
train = shuffled.take(train_size)
test = shuffled.skip(train_size)

In [45]:
cached_train = train.shuffle(100_000).batch(2048)
cached_test = test.batch(4096).cache()

In [46]:
num_epochs = 40
model = deep_postlevelModel([32])
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
one_layer_history = model.fit(
    cached_train,
    validation_data=cached_test,
    validation_freq=5,
    epochs=num_epochs)
accuracy = one_layer_history.history["val_factorized_top_k/top_100_categorical_accuracy"][-1]
print(f"Accuracy: {accuracy:.2f}.")

Epoch 1/40
11/11 [==============================] - 10s 647ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0192 - factorized_top_k/top_5_categorical_accuracy: 0.0260 - factorized_top_k/top_10_categorical_accuracy: 0.0303 - factorized_top_k/top_50_categorical_accuracy: 0.0429 - factorized_top_k/top_100_categorical_accuracy: 0.0531 - loss: 15873.6353 - regularization_loss: 0.0000e+00 - total_loss: 15873.6353
Epoch 2/40
11/11 [==============================] - 8s 696ms/step - factorized_top_k/top_1_categorical_accuracy: 4.0014e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0017 - factorized_top_k/top_10_categorical_accuracy: 0.0034 - factorized_top_k/top_50_categorical_accuracy: 0.0152 - factorized_top_k/top_100_categorical_accuracy: 0.0283 - loss: 15155.9976 - regularization_loss: 0.0000e+00 - total_loss: 15155.9976
Epoch 3/40
11/11 [==============================] - 9s 854ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0013 - factorized_top_k/top_5_categorical_ac

In [47]:
num_epochs = 40
model = deep_postlevelModel([64,32])
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
two_layer_history = model.fit(
    cached_train,
    validation_data=cached_test,
    validation_freq=5,
    epochs=num_epochs)
accuracy = two_layer_history.history["val_factorized_top_k/top_100_categorical_accuracy"][-1]
print(f"Accuracy: {accuracy:.2f}.")

Epoch 1/40
11/11 [==============================] - 9s 675ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0188 - factorized_top_k/top_5_categorical_accuracy: 0.0381 - factorized_top_k/top_10_categorical_accuracy: 0.0520 - factorized_top_k/top_50_categorical_accuracy: 0.1274 - factorized_top_k/top_100_categorical_accuracy: 0.1823 - loss: 19002.4803 - regularization_loss: 0.0000e+00 - total_loss: 19002.4803
Epoch 2/40
11/11 [==============================] - 8s 722ms/step - factorized_top_k/top_1_categorical_accuracy: 9.7813e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0036 - factorized_top_k/top_10_categorical_accuracy: 0.0060 - factorized_top_k/top_50_categorical_accuracy: 0.0239 - factorized_top_k/top_100_categorical_accuracy: 0.0425 - loss: 15575.7938 - regularization_loss: 0.0000e+00 - total_loss: 15575.7938
Epoch 3/40
11/11 [==============================] - 10s 941ms/step - factorized_top_k/top_1_categorical_accuracy: 8.8921e-05 - factorized_top_k/top_5_categorica

In [48]:
num_epochs = 40
model = deep_postlevelModel([64,32])
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
three_layer_history = model.fit(
    cached_train,
    validation_data=cached_test,
    validation_freq=5,
    epochs=num_epochs)
accuracy = three_layer_history.history["val_factorized_top_k/top_100_categorical_accuracy"][-1]
print(f"Accuracy: {accuracy:.2f}.")

Epoch 1/40
11/11 [==============================] - 12s 869ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0498 - factorized_top_k/top_5_categorical_accuracy: 0.0612 - factorized_top_k/top_10_categorical_accuracy: 0.0666 - factorized_top_k/top_50_categorical_accuracy: 0.0803 - factorized_top_k/top_100_categorical_accuracy: 0.0934 - loss: 19323.0151 - regularization_loss: 0.0000e+00 - total_loss: 19323.0151
Epoch 2/40
11/11 [==============================] - 10s 872ms/step - factorized_top_k/top_1_categorical_accuracy: 4.4460e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0015 - factorized_top_k/top_10_categorical_accuracy: 0.0026 - factorized_top_k/top_50_categorical_accuracy: 0.0107 - factorized_top_k/top_100_categorical_accuracy: 0.0261 - loss: 15568.7425 - regularization_loss: 0.0000e+00 - total_loss: 15568.7425
Epoch 3/40
11/11 [==============================] - 9s 837ms/step - factorized_top_k/top_1_categorical_accuracy: 2.2230e-04 - factorized_top_k/top_5_categoric

In [49]:
num_epochs = 40

model = deep_postlevelModel([64,32])
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
four_layer_history = model.fit(
    cached_train,
    validation_data=cached_test,
    validation_freq=5,
    epochs=num_epochs)

accuracy = three_layer_history.history["val_factorized_top_k/top_100_categorical_accuracy"][-1]
print(f"Accuracy: {accuracy:.2f}.")


Epoch 1/40
11/11 [==============================] - 18s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0682 - factorized_top_k/top_5_categorical_accuracy: 0.0928 - factorized_top_k/top_10_categorical_accuracy: 0.1054 - factorized_top_k/top_50_categorical_accuracy: 0.1234 - factorized_top_k/top_100_categorical_accuracy: 0.1320 - loss: 17093.5765 - regularization_loss: 0.0000e+00 - total_loss: 17093.5765
Epoch 2/40
11/11 [==============================] - 18s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0026 - factorized_top_k/top_5_categorical_accuracy: 0.0049 - factorized_top_k/top_10_categorical_accuracy: 0.0072 - factorized_top_k/top_50_categorical_accuracy: 0.0185 - factorized_top_k/top_100_categorical_accuracy: 0.0294 - loss: 15557.8401 - regularization_loss: 0.0000e+00 - total_loss: 15557.8401
Epoch 3/40
11/11 [==============================] - 18s 2s/step - factorized_top_k/top_1_categorical_accuracy: 7.1136e-04 - factorized_top_k/top_5_categorical_accuracy: